# Genetic Algorithm - 2D Example

In [ ]:
# objective functions
import numpy as np


def f(x, y):
    omega = 1.6
    sigma = 20.0
    return (np.sin(omega * x) ** 2) * (np.sin(omega * y) ** 2) * np.exp(
        -(x + y) / sigma
    ) + 10.0

In [ ]:
# plot objective function
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from IPython import get_ipython

get_ipython().run_line_magic("matplotlib", "widget")

# sample points in a mesh
delta = 0.1
x = np.arange(0.0, 10.0, delta)
y = np.arange(0.0, 10.0, delta)
X, Y = np.meshgrid(x, y)
Z = f(X, Y)

# create matplotlib plot
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection="3d")
ax.plot_surface(
    X, Y, Z, rstride=1, cstride=1, cmap="coolwarm", linewidth=0, antialiased=False
)
plt.show()

In [ ]:
## conversion functions
def bin_to_int(b):
    return int(b, 2)


def bin_to_2D_float(b, lb=0.0, ub=10.0, print_all=False):
    """Converts the two halves of a binary string to two floating point
    numbers using linear mapping within given bounds for the numbers.

    Args:
        b (str): binary string
        lb (float, optional): lower bound. Defaults to 0.0.
        ub (float, optional): upper bound. Defaults to 10.0.
        print_all (bool, optional): print process. Defaults to False.

    Returns:
        [float]: tuple of two floating point numbers
    """
    i1 = ""
    i2 = ""
    for idx in range(len(b)):
        if idx < len(b) / 2:
            i1 += b[idx]
        else:
            i2 += b[idx]

    Max_i = "1" * len(i1)
    val1 = ((ub - lb) / float(int(Max_i, 2))) * float(int(i1, 2)) + lb
    val2 = ((ub - lb) / float(int(Max_i, 2))) * float(int(i2, 2)) + lb
    if print_all:
        print(
            "string is split into {} ({}) and {} ({})".format(
                i1, bin_to_int(i1), i2, bin_to_int(i2)
            )
        )
        print(f"The corresponding values are: {val1}, {val2}")
    return (val1, val2)

In [ ]:
vals = bin_to_2D_float("101010", lb=0, ub=10, print_all=True)

# Operators used by Genetic algorithm

In [ ]:
def initial_population(n):
    """
    Creates the initial population with 'n' individuals
    """
    sl = 60  # Length of the binary string of the individuals
    generation = []
    for i in range(n):
        individual = ""
        for j in range(sl):
            individual += str(np.random.choice([0, 1]))

        generation.append(individual)
    return generation

In [ ]:
print("Initial Population: \n", initial_population(n=4))

# 2D Objective functions

In [ ]:
def f1(x):
    return f(x[0], x[1])


fitness = f1

In [ ]:
def roulette_wheel(G, ax):
    """
    Returns a roulette wheel for a given population 'G' according to the
    fitness value of the individuals, 'ax' is an Axes object on which the
    roulette wheel is plotted
    """
    fracs = []
    wheel = []
    labels = []
    color_vals = [i for i in range(len(G))]
    norm = colors.Normalize(0, len(G))
    cmap = cm.get_cmap("jet")
    color = cmap(norm(color_vals))
    Gx = [bin_to_2D_float(i) for i in G]
    fGx = [fitness(i) for i in Gx]
    for i in range(len(G)):
        labels.append("Ind-" + str(i))
        fracs.append(float(fGx[i]) / float(sum(fGx)))
    for i in range(len(fracs)):
        if i == 0:
            wheel.append(fracs[i])
        else:
            wheel.append(wheel[i - 1] + fracs[i])
    ax.pie(fracs, labels=labels, colors=color, normalize=True)
    return wheel

In [ ]:
# plot an example roulette wheel
fig, ax = plt.subplots()
G0 = initial_population(5)
for i in range(len(G0)):
    print("Fitness of individual {} is {}".format(i, fitness(bin_to_2D_float(G0[i]))))
W0 = roulette_wheel(G0, ax)
print("Wheel (CDF): ", W0)
plt.show()

In [ ]:
def mating(G, W):
    """
    Returns a mating pool. For each individual in the population 'G', the
    roulette wheel 'W' is turned once from which the id of a mate is selected
    """
    Mates = []
    for _ in G:
        test = np.random.rand()
        for i in range(len(W)):
            if i == 0:
                if test <= W[i] and test > 0.0:
                    mate = i
            else:
                if test <= W[i] and test > W[i - 1]:
                    mate = i
        Mates.append(mate)
    return Mates

In [ ]:
def crossover(G, M1, M2, probability=1.0, keep=1, print_all=False):
    """
    Performs one-point crossover for a population 'G', two parents are selected
    using two mating pools 'M1' and 'M2'. Crossover is performed according to
    the 'probability' variable, if probability = 1.0, crossover is performed
    for every individual in the population, if probability = 0.0, crossover
    is not performed, i.e. the closer the value of the probability is to zero,
    crossover is less likely to happen, the closer to 1.0, crossover is most
    likely to occur. Only one offspring is kept to form the new generation,
    the 'keep' variable controls this decision, keep can be 1, 2 or 'random',
    1 for keeping the first offspring, 2 the second, and 'random' to decide
    with a coin toss.
    """
    New_Generation = []

    assert keep in ["random", 1, 2]

    if print_all:
        print(
            "|-{1:->12}---{2:->12}---{3:->12}---{4:->12}-|".format("", "", "", "", "")
        )
        print(
            "| {1:>12}   {2:>12}   {3:>12}   {4:>12} |".format(
                "", "Crossover:", "", "", ""
            )
        )
        print(
            "|-{1:->12}---{2:->12}---{3:->12}---{4:->12}-|".format("", "", "", "", "")
        )
        print(
            "| {1:^12} | {2:^12} | {3:^12} | {4:^12} |".format(
                "", "Id", "Parents", "Child", "Keep"
            )
        )
        print(
            "|-{1:->12}---{2:->12}---{3:->12}---{4:->12}-|".format("", "", "", "", "")
        )

    for i in range(len(G)):
        # figure out where to crossover
        if np.random.rand() < probability:
            cp = np.random.randint(1, len(G[i]) - 1)  # cp .. cross point
        else:
            cp = 0
        Parent_1 = G[M1[i]][:cp] + "|" + G[M1[i]][cp:]
        Parent_2 = G[M2[i]][:cp] + "|" + G[M2[i]][cp:]
        Child_1 = G[M1[i]][:cp] + "|" + G[M2[i]][cp:]
        Child_2 = G[M2[i]][:cp] + "|" + G[M1[i]][cp:]
        Children = [Child_1, Child_2]

        # remove the divider
        if keep == "random":
            keep = np.random.choice([1, 2])

        # decide which child to keep (no pun intended)
        kept = Children[keep - 1].replace("|", "")

        New_Generation.append(kept)
        if print_all:
            print(
                "| {1:>12} | {2:>12} | {3:>12} | {4:>12} |".format(
                    "", "Ind-" + str(M1[i]), Parent_1, Child_1, kept
                )
            )
            print(
                "| {1:>12} | {2:>12} | {3:>12} | {4:>12} |".format(
                    "", "Ind-" + str(M2[i]), Parent_2, Child_2, ""
                )
            )
            print(
                "|-{1:->12}---{2:->12}---{3:->12}---{4:->12}-|".format(
                    "", "", "", "", ""
                )
            )
    return New_Generation

In [ ]:
def mutate(G, probability=0.001, print_all=False):
    """
    Performs mutation for every individual in the population 'G' according to
    the 'probability' variable.
    """
    New_Generation = []
    if print_all:
        print("|-{0:->12}---{1:->12}---{2:->12}-|".format("", "", ""))
        print("| {0:>12}   {1:>12}   {2:>12} |".format("Mutation:", "", ""))
        print("|-{0:->12}---{1:->12}---{2:->12}-|".format("", "", ""))
        print("| {0:^12} | {1:^12} | {2:^12} |".format("Id", "Original", "New"))
        print("|-{0:->12}---{1:->12}---{2:->12}-|".format("", "", ""))
    for i, Ind in enumerate(G):
        New_Ind = ""
        for ch in Ind:
            if np.random.rand() < probability:
                if ch == "1":
                    new_ch = "0"
                if ch == "0":
                    new_ch = "1"
                New_Ind += new_ch
            else:
                New_Ind += ch
        New_Generation.append(New_Ind)
        if print_all:
            print(
                "| {0:^12} | {1:^12} | {2:^12} |".format("Ind-" + str(i), Ind, New_Ind)
            )
    if print_all:
        print("|-{0:->12}---{1:->12}---{2:->12}-|".format("", "", ""))
    return New_Generation

In [ ]:
def print_fitness(G):
    # calculate fitness
    Gx = [bin_to_2D_float(i) for i in G]
    fGx = [fitness(i) for i in Gx]

    print("|-{0:->12}---{1:->45}---{2:->12}-|".format("", "", "", ""))
    print("| {0:^12}| {1:^45} | {2:^12} |".format("Id", "x Value", "Fitness"))
    print("|-{0:->12}---{1:->45}---{2:->12}-|".format("", "", ""))
    for i in range(len(G)):
        print(
            "| {0:^12} | ({1:^20} , {2:^20}) | {3:^12.2f} |".format(
                "Ind-" + str(i), Gx[i][0], Gx[i][1], fGx[i]
            )
        )
    print("|-{0:->12}---{1:->45}---{2:->12}-|".format("", "", ""))
    print("| {0:<12}   {1:>45} | {2:12.2f} |".format("Sum", "", sum(fGx)))
    print(
        "| {0:<12}   {1:>45} | {2:12.2f} |".format(
            "Average", "", sum(fGx) / float(len(fGx))
        )
    )
    print("| {0:<12}   {1:>45} | {2:12.2f} |".format("Max", "", max(fGx)))
    print("|-{0:->12}---{1:->45}---{2:->12}-|".format("", "", ""))

In [ ]:
G0 = initial_population(4)
print_fitness(G0)

In [ ]:
import itertools


def plot_population(G, ax, alpha=0.7):
    Gx = [bin_to_2D_float(i) for i in G]
    color_iterator = itertools.cycle(colors.BASE_COLORS.keys())
    for i in range(len(Gx)):
        ax.scatter(Gx[i][0], Gx[i][1], c=next(color_iterator), alpha=alpha)

In [ ]:
# show fitness of the population
fig, ax = plt.subplots(figsize=(6, 4))
ax.set_xlim([0, 10])
ax.set_ylim([0, 10])
ax.grid()
ax.set_xlabel("x")
ax.set_ylabel("y", rotation=0)
delta = 0.1
x = np.arange(0.0, 10.0, delta)
y = np.arange(0.0, 10.0, delta)
X, Y = np.meshgrid(x, y)
Z = f(X, Y)
ax.contour(X, Y, Z, cmap=cm.coolwarm, antialiased=False)
G0 = initial_population(40)
plot_population(G0, ax)
plt.show()

### Change population size (default = 50), mutation probability (default = 0.3)
1. Change population size from 50 to 10
2. Change population size from 50 to 100
3. 1/l heuristic: Change mutation level to 1/l where l is length of individual string.

Exponential relation between absorption and the string length ( O($e^l$) ). So, high mutations can be disastrous for Genetric algorithms

In [ ]:
# initialize and iteration limit

n = 50  # population size
G = initial_population(n)
t = 0
t_max = 50
# set plot layout
delta = 0.1
x = np.arange(0.0, 10.0, delta)
y = np.arange(0.0, 10.0, delta)
X, Y = np.meshgrid(x, y)
Z = f(X, Y)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

In [ ]:
# run evalution algorithm
import time

for t in range(t_max):
    # set up axes
    # clear axes
    ax1.clear()
    ax2.clear()

    fig.suptitle("Iteration = {0:>3}".format(t))
    ax2.set_aspect("equal")
    ax1.set_xlim([0, 10])
    ax1.set_ylim([0, 10])
    ax1.grid()
    ax1.set_xlabel("x")
    ax1.set_ylabel("y", rotation=0)

    # plot
    ax1.contour(X, Y, Z, cmap=cm.coolwarm, antialiased=False)
    plot_population(G, ax1, alpha=0.5)

    # prepare new generation
    W = roulette_wheel(G, ax2)
    M1 = mating(G, W)
    M2 = mating(G, W)
    C = crossover(G, M1, M2, probability=0.7, keep="random")  # default: 0.7
    G = mutate(C, probability=0.05)  # default: 0.05

    # redraw plots
    fig.canvas.draw()
    fig.canvas.flush_events()
    time.sleep(0.01)

print_fitness(G)

In [ ]:
# Increase selective pressure
def f2(x):
    return f(x[0], x[1]) ** 4


fitness = f2

In [ ]:
# Bias the objective to improve variation and increase selective pressure
# You should observe much better performance
# So design your objective carefully!
def f3(x):
    return ((f(x[0], x[1]) - 10.0)) ** 4


fitness = f3